In [1]:
import torch
from PIL import Image
import requests
import torchvision.transforms as T
import matplotlib.pyplot as plt
from collections import defaultdict
import torch.nn.functional as F
import numpy as np
from skimage.measure import find_contours

from matplotlib import patches,  lines
from matplotlib.patches import Polygon

torch.set_grad_enabled(False);

In [2]:
%matplotlib notebook
%matplotlib inline

VQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/VQA'
GQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/GQA'

import pandas as pd
import json
import pickle as pkl
import numpy as np
import h5py
import random
from PIL import Image
import IPython.display as display
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import torch

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [3]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [4]:
import inflect
stemmer = inflect.engine()

In [5]:
gqa_val_q=json.load(
    open(os.path.join(GQA_PATH,'original','val_balanced_questions.json')
         ,'r')
)
print (len(gqa_val_q))
names=list(gqa_val_q.keys())

val_graphs=json.load(
    open(os.path.join(GQA_PATH,'val_sceneGraphs.json')
         ,'r')
)
print (len(val_graphs))

132062
10696


In [6]:
import skimage
from skimage import io as skimage_io
from skimage import transform as skimage_transform

def get_tokens(text_queries):
    tokenized_queries = np.array([
        module.tokenize(q, config.dataset_configs.max_query_length)
        for q in text_queries
    ])
    # Pad tokenized queries to avoid recompilation if number of queries changes:
    tokenized_queries = np.pad(
        tokenized_queries,
        pad_width=((0, 100 - len(text_queries)), (0, 0)),
        constant_values=0)
    return tokenized_queries

def get_gqa_feat(img_id):
    # Load example image:
    filename = os.path.join(GQA_PATH,'images',img_id+'.jpg')
    image_uint8 = skimage_io.imread(filename)
    image = image_uint8.astype(np.float32) / 255.0

    # Pad to square with gray pixels on bottom and right:
    h, w, _ = image.shape
    size = max(h, w)
    image_padded = np.pad(
        image, ((0, size - h), (0, size - w), (0, 0)), constant_values=0.5)

    # Resize to model input size:
    input_image = skimage.transform.resize(
        image_padded,
        (840, 840),
        anti_aliasing=True)
    return input_image

In [7]:
import clip

CUDA_DEVICE=4
torch.cuda.set_device(CUDA_DEVICE)
device = torch.device("cuda:"+str(CUDA_DEVICE))
#the default ipykernel links to the first conda environment

model, postprocessor = torch.hub.load('ashkamath/mdetr:main', 'mdetr_efficientnetB5', pretrained=True, return_postprocessor=True)
model = model.to(device)
model.eval();

clip_model, preprocess = clip.load("ViT-B/32",device=device)

Using cache found in /root/.cache/torch/hub/ashkamath_mdetr_main
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def plot_inference(im, caption, plot=True):
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0).cuda()

    # propagate through the model
    memory_cache = model(img, [caption], encode_and_save=True)
    outputs = model(img, [caption], encode_and_save=False, memory_cache=memory_cache)
    #print (outputs['pred_logits'])

    # keep only predictions with 0.7+ confidence
    probas = 1 - outputs['pred_logits'].softmax(-1)[0, :, -1].cpu()
    keep = (probas > 0.7).cpu()

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'].cpu()[0, keep], im.size)

    # Extract the text spans predicted by each box
    positive_tokens = (outputs["pred_logits"].cpu()[0, keep].softmax(-1) > 0.1).nonzero().tolist()
    predicted_spans = defaultdict(str)
    for tok in positive_tokens:
        item, pos = tok
        if pos < 255:
            span = memory_cache["tokenized"].token_to_chars(0, pos)
            predicted_spans [item] += " " + caption[span.start:span.end]

    labels = [predicted_spans [k] for k in sorted(list(predicted_spans .keys()))]
    if plot:
        plot_results(im, probas[keep], bboxes_scaled, labels)
    return probas[keep], bboxes_scaled.tolist(), labels

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

def plot_results(pil_img, scores, boxes, labels, masks=None):
    plt.figure(figsize=(16,10))
    np_image = np.array(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if masks is None:
        masks = [None for _ in range(len(scores))]
    assert len(scores) == len(boxes) == len(labels) == len(masks)
    for s, (xmin, ymin, xmax, ymax), l, mask, c in zip(scores, boxes.tolist(), labels, masks, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{l}: {s:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

        if mask is None:
            continue
        np_image = apply_mask(np_image, mask, c)

        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=c)
            ax.add_patch(p)


    plt.imshow(np_image)
    plt.axis('off')
    plt.show()


def add_res(results, ax, color='green'):
    #for tt in results.values():
    if True:
        bboxes = results['boxes']
        labels = results['labels']
        scores = results['scores']
        #keep = scores >= 0.0
        #bboxes = bboxes[keep].tolist()
        #labels = labels[keep].tolist()
        #scores = scores[keep].tolist()
    #print(torchvision.ops.box_iou(tt['boxes'].cpu().detach(), torch.as_tensor([[xmin, ymin, xmax, ymax]])))
    
    colors = ['purple', 'yellow', 'red', 'green', 'orange', 'pink']
    
    for i, (b, ll, ss) in enumerate(zip(bboxes, labels, scores)):
        ax.add_patch(plt.Rectangle((b[0], b[1]), b[2] - b[0], b[3] - b[1], fill=False, color=colors[i], linewidth=3))
        cls_name = ll if isinstance(ll,str) else CLASSES[ll]
        text = f'{cls_name}: {ss:.2f}'
        print(text)
        ax.text(b[0], b[1], text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

In [9]:
def vis_bbox_only(bbox,scores,input_image,gt_bbox=None,text=None,threshold=0.2,vis_pred=True):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(input_image, extent=(0, 1, 1, 0))
    ax.set_axis_off()
    if vis_pred:
        for i,box in enumerate(bbox):
            score=scores[i]
            if score<threshold:
                continue
            cx, cy, w, h = box
            ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                    [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'r')
            ax.text(
                cx - w / 2,
                cy + h / 2 + 0.015,
                f'{score:1.2f}',
                ha='left',
                va='top',
                color='red',
                bbox={
                    'facecolor': 'white',
                    'edgecolor': 'red',
                    'boxstyle': 'square,pad=.3'
                })
    if text is not None:
        ax.set_title(text,  fontsize=12)
    if gt_bbox is not None:
        for i,box in enumerate(gt_bbox):
            cx, cy, w, h = box
            ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                    [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'b')

In [10]:
import IPython.display as display

def size_aware_pred(bb):
    if bb[2]>0.3 or bb[3]>0.3:
        return 'large'
    else:
        return 'small'
def pos_aware_pred(valid_bbox,candidates):
    if 'bottom' in candidates:
        if valid_bbox[1]>0.5:
            return 'bottom'
        else:
            return 'top'
    elif 'right' in candidates:
        if valid_bbox[0]<0.5:
            return 'left'
        else:
            return 'right'

In [11]:
def transform_coord(bbox,img_id):
    width=val_graphs[img_id]['width']
    height=val_graphs[img_id]['height']
    size = max(height, width)
    x=bbox[0]
    y=bbox[1]
    w=bbox[2]-bbox[0]
    h=bbox[3]-bbox[1]
    #print (bbox)
    return [(x+w/2)/size,(y+h/2)/size,w/size,h/size]

In [12]:
def get_adjectives(text):
    sent=text.split(',')[0]
    words=sent.split(' ')
    if 'less' in words:
        words=words[-2:]
    else:
        words=words[-1:]
    words=' '.join(words)
    return words

In [13]:
def preprocess_mdert_result(scores,bboxs,labels,obj,img_id):
    sf_s=[]
    sf_bbox=[]
    labels=[''.join(label.split(' ')) for label in labels]
    flag=False
    if obj in labels:
        flag=True
    #print (obj,labels)
    for i,label in enumerate(labels):
        if flag:
            if obj==label:
                sf_s.append(scores[i])
                sf_bbox.append(bboxs[i])
        else:
            if obj in label:
                sf_s.append(scores[i])
                sf_bbox.append(bboxs[i])
    #print (len(sf_s))
    if len(sf_bbox)==0:
        for i,label in enumerate(labels):
            sf_s.append(scores[i])
            sf_bbox.append(bboxs[i])
    max_s=max(sf_s)
    max_id=sf_s.index(max_s)
    return_bbox=sf_bbox[max_id]
    coord=transform_coord(return_bbox,img_id)
    return max_s.item(),coord
#only consider one bbox now

In [14]:
def generate_phrase_ground(img_id,cap,obj):
    cap=cap+' '
    im=Image.open(os.path.join(GQA_PATH,'images',img_id+'.jpg'))
    try:
        scores,bboxs,labels=plot_inference(im, cap, plot=False)
    except:
        print ('Invalid caption generation for',img_id,obj,cap)
        scores=[torch.Tensor([0.99])]
        bboxs=[[0.5,0.5,0.99,0.99]]
        labels=[obj]
    if len(bboxs)==0:
        max_s=0.99
        valid_bbox=[0.5,0.5,0.99,0.99]
    else:
        # print (scores,bboxs)
        max_s,valid_bbox=preprocess_mdert_result(scores,bboxs,labels,obj,img_id)
    return max_s,valid_bbox

def bbox_generator(layout,idx,img_id,ques_id):
    if layout[idx]['operation']=='select':
        #print (idx)
        try:
            scenic_result=load_pkl(os.path.join(GQA_PATH,
                                                'meta_one-all-scenic',
                                                ques_id+'.pkl'))
            obj=layout[idx]['argument'][0]
            scores=[s['score'] for s in scenic_result[obj]]
            bboxs=[s['bbox'] for s in scenic_result[obj]]
        except:
            bboxs=[]
            obj=layout[idx]['argument'][0]
        if len(bboxs)>0:
            max_s=max(scores)
            max_idx=scores.index(max_s)
            bbox=bboxs[max_idx]
            cap=obj
        else:
            cap=obj
            split_obj=''.join(obj.split(' '))
            #print (cap,split_obj)
            max_s,bbox=generate_phrase_ground(img_id,cap,split_obj)
            #print (max_s,bbox)
    else:
        cap,obj=cap_generator(layout,idx,img_id)
        max_s,bbox=generate_phrase_ground(img_id,cap,obj)
    return max_s,bbox,cap        
            
def cap_generator(layout,idx,img_id):
    words=[]
    dep=layout[idx]['dependencies']
    word=layout[idx]['argument'][0]
    words.append(word)
    obj=''.join(word.split(' '))
    while len(dep)>0:
        cur_step=layout[dep[0]]
        if cur_step['operation']=='relocate':
            relo_symbol=cur_step['argument'][1]
            cur_phrase=' '.join(words)
            words=[]
            words.append(cur_phrase)
            #print(cur_step)
            words.append(cur_step['argument'][0])
            dep=cur_step['dependencies']
            cur_step=layout[dep[0]]
            dep=cur_step['dependencies']
            arg=cur_step['argument'][0]
            words.append(arg)
            if relo_symbol=='o':
                words=list(reversed(words))
        else:
            arg=cur_step['argument'][0]
            words.append(arg)
            dep=cur_step['dependencies']
    cap=' '.join(words)
    return cap,obj

In [15]:
def phrase_bbox_generator_exist(layout,idx,img_id):
    cap=cap_generator_exist(layout,idx,img_id)
    max_s,bbox=generate_phrase_ground(img_id,cap,cap)
    return bbox,cap

In [16]:
def get_cropped(bb,input_image,showing=False):
    width=bb[2]*840
    height=bb[3]*840
    xy=(bb[0]*840-width/2,bb[1]*840-height/2)
    h0=int(max(0,xy[1]))
    w0=int(max(0,xy[0]))
    h1=int(max(0,840-(height+xy[1])))
    w1=int(max(0,840-(width+xy[0])))
    cropped=skimage.util.crop(input_image,((h0,h1),(w0,w1),(0,0)), copy=False)
    trans_crop=Image.fromarray(np.uint8(cropped*255.0))
    if showing:
        display.display(trans_crop)
    return trans_crop

def clip_aware_pred(img_feat,valid_bbox,candidates,showing=False):
    #tokens=clip.tokenize(candidates)
    trans_crop=get_cropped(valid_bbox,img_feat)
    if showing:
        display.display(trans_crop)
    with torch.no_grad():
        image_features = clip_model.encode_image(preprocess(trans_crop).unsqueeze(0).to(device))
        text_features = clip_model.encode_text(clip.tokenize(candidates).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity.topk(2)
    #print (indices[0][0].item())
    ans=candidates[indices[0][0].item()]
    return ans

In [17]:
position_holder=['left','right','bottom','top','front']

In [18]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [19]:
def get_nouns(ques):
    nouns=[]
    tokens_tag = pos_tag(ques.replace('?','').split(' '))
    for w,pos in tokens_tag:
        if pos in ['NN','NNS'] and w not in position_holder:
            nouns.append(lemmatizer.lemmatize(w))
    return nouns

def verify_noun(nouns,obj):
    if obj in nouns or \
    lemmatizer.lemmatize(obj) in nouns or\
    stemmer.singular_noun(obj) in nouns:
        return True
    elif len(obj.split(' '))>1:
        words=obj.split(' ')
        for w in words:
            #print (w)
            if lemmatizer.lemmatize(w) in nouns or\
            stemmer.singular_noun(w) in nouns:
                return True
    elif obj in ['he','she','they']:
        return True
    return False

In [20]:
def get_relo(layout):
    ops=[step['operation'] for step in layout]
    length=len(layout)-2
    relo_id=length
    counter=0
    rela_symbol=[]
    for i in range(len(layout)-2):
        cur_op=layout[length-i]['operation']
        if cur_op=='relocate':
            relo_id=length-i
            rela_symbol=layout[length-i]['argument']
            break
        elif cur_op=='filter':
            counter+=1#find some attributes finally, if true
    return rela_symbol,relo_id, counter

In [21]:
def cap_generator_exist(layout,idx,img_id):
    words=[]
    dep=layout[idx]['dependencies']
    while len(dep)>0:
        cur_step=layout[dep[0]]
        if cur_step['operation']=='relocate':
            relo_symbol=cur_step['argument'][1]
            cur_phrase=' '.join(words)
            words=[]
            words.append(cur_phrase)
            #print(cur_step)
            words.append(cur_step['argument'][0])
            dep=cur_step['dependencies']
            new_sent=[]
            while len(dep)>0:
                cur_step=layout[dep[0]]
                dep=cur_step['dependencies']
                arg=cur_step['argument'][0]
                new_sent.append(arg)
            words.append(' '.join(new_sent))
            if relo_symbol=='o':
                words=list(reversed(words))
        else:
            arg=cur_step['argument'][0]
            words.append(arg)
            dep=cur_step['dependencies']
    cap=' '.join(words)
    return cap

In [22]:
def relocate_involved_ans(layout,img_id,ques_id,thresh=0.2):
    nouns=get_nouns(gqa_val_q[ques_id]['question'][:-1])#remove the punctuation
    ops=[step['operation'] for step in layout]
    #must have select operation
    for m,step in enumerate(layout[:-1]):
        op=step['operation']
        if op not in ['filter','select']:
            continue
        arg=step['argument'][0]
        if arg in hold_list:
            continue
        if op=='filter':
            flag=verify_noun(nouns,arg)
            if flag==False:#not a noun
                continue
        if op=='select':
            #infos=select_info[arg]
            s,_,_=bbox_generator(layout,m,img_id,ques_id)
            if s<thresh:
                return 'no'
        else:
            try:
                filter_info=load_pkl(os.path.join(GQA_PATH,
                                              'meta_filter-related',
                                              ques_id+'.pkl'))
                infos=filter_info[arg]
                s=[info['score'] for info in infos if info['score']>=thresh]
            except:
                print('No filter result',arg,ques_id)
                continue
            
            if len(s)==0:
                return 'no'
    rela_symbol,rela_id,counter=get_relo(new_prog)    
    if counter==1:
        pred=verify_relation(layout,img_id,ques_id,rela_symbol,rela_id)
    else:
        pred=verify_attr_last_step(layout,img_id,ques_id)
    return pred

In [23]:
heur_rela=['to the left of','to the right of',
           'above','under','on top of','below',
           'beneath','underneath']
#if color on the last argument, use phrase grounding and binary color function

In [24]:
def heur_rules(bb_a,bb_b,rela):
    #whether there is bb_a rela bb_b
    if rela=='to the left of':
        if bb_a[0]<bb_b[0]:
            return 'yes'
        else:
            return 'no'
    elif rela=='to the right of':
        if bb_a[0]>bb_b[0]:
            return 'yes'
        else:
            return 'no'
    elif rela in ['above','on top of']:
        if bb_a[1]>bb_b[1]:
            return 'yes'
        else:
            return 'no'
    elif rela in [ 'under','below','beneath','underneath']:
        if bb_a[1]<bb_b[1]:
            return 'yes'
        else:
            return 'no' 

In [25]:
def verify_relation(layout,img_id,ques_id,rela_symbol,rela_id,showing=False):
    relation=rela_symbol[0]
    sub_obj=rela_symbol[1]
    obj=layout[-2]['argument'][0]
    cap=cap_generator_exist(layout,len(layout)-1,img_id)
    cap=cap.replace(obj,'').replace(relation,'')
    if relation in heur_rela:
        try:
            scenic_info=load_pkl(os.path.join(GQA_PATH,
                                              'meta_filter-related',
                                              ques_id+'.pkl'))
        #print (scenic_info.keys())
            info=scenic_info[obj]
            scores=[i['score'] for i in info]
            bboxs=[i['bbox'] for i in info]
            if len(scores)==0:
                return 'no'
            bb_1=bboxs[scores.index(max(scores))]
        except:
            s,b,_=bbox_generator(layout,len(layout)-2,img_id,ques_id)
            bb_1=b
        
        if rela_id==1:
            _,bb_2,_=bbox_generator(layout,0,img_id,ques_id)
        else:
            obj=layout[rela_id-1]['argument'][0]
            _,bb_2=generate_phrase_ground(img_id,cap,obj)
        if sub_obj=='o':
            bb_a=bb_2
            bb_b=bb_1
        else:
            bb_a=bb_1
            bb_b=bb_2
        pred=heur_rules(bb_a,bb_b,relation)
        if showing:
            img_feat=get_gqa_feat(img_id)
            #print (bb_a,bb_b)
            vis_bbox_only([bb_a],[0.5],img_feat,[bb_b],img_id)
    else:
        if sub_obj=='s':
            pos=' '.join([obj,'is',relation,cap])
            neg=' '.join([obj,'is not',relation,cap])
        else:
            pos=' '.join([cap,'is',relation,obj])
            neg=' '.join([cap,'is not',relation,obj])
        #print (pos,'\n',neg,'\n',gqa_val_q[ques_id]['question'])
        pred=binary_matching(img_id,[pos,neg])
    return pred

In [26]:
def binary_matching(img_id,sents):
    tokens=clip.tokenize(sents)
    im=Image.open(os.path.join(GQA_PATH,'images',img_id+'.jpg'))
    with torch.no_grad():
        text_features=clip_model.encode_text(tokens.to(device))
        image_features = clip_model.encode_image(preprocess(im).unsqueeze(0).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    indices=torch.max(similarity,1)[1]
    #print (similarity,indices)
    similarity=similarity.squeeze()
    scores=[str(similarity[0].item()),str(similarity[1].item())]
    if indices.squeeze().item()==0:
        pred='yes'
    else:
        pred='no'
    return pred

In [27]:
def verify_attr_last_step(layout,img_id,ques_id,showing=False):
    cap=cap_generator_exist(layout,len(layout)-1,img_id)
    attr=layout[-2]['argument'][0]
    phrase=cap.replace(attr,'')
    obj=layout[-3]['argument'][0]
    _,bbox=generate_phrase_ground(img_id,phrase,obj)
    input_image=get_gqa_feat(img_id)
    pred=filter_binary_color(bbox,input_image,attr,showing)
    return pred

In [28]:
def filter_binary_color(bb,input_image,color,showing=False):
    width=bb[2]*840
    height=bb[3]*840
    xy=(bb[0]*840-width/2,bb[1]*840-height/2)
    h0=int(max(0,xy[1]))
    w0=int(max(0,xy[0]))
    h1=int(max(0,840-(height+xy[1])))
    w1=int(max(0,840-(width+xy[0])))
    cropped=skimage.util.crop(input_image,((h0,h1),(w0,w1),(0,0)), copy=False)
    trans_crop=Image.fromarray(np.uint8(cropped*255.0))
    flag=True #color is a positive statement
    if 'not' in color:
        flag=False#reverse the pred
        words=color.split(' ')[1:]
        color=' '.join(words)
    pos_sent=color
    neg_sent='Not '+color
    tokens=clip.tokenize([neg_sent,pos_sent])
    with torch.no_grad():
        text_features=clip_model.encode_text(tokens.to(device))
        image_features = clip_model.encode_image(preprocess(trans_crop).unsqueeze(0).to(device))
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    indices=torch.max(similarity,1)[1]
    #print (similarity,indices)
    similarity=similarity.squeeze()
    scores=[str(similarity[0].item()),str(similarity[1].item())]
    if showing:
        vis_no_score([bb],input_image,'not '+color+'-'+'-'.join(scores))
    if indices.squeeze().item()==1:
        pred='yes'
    else:
        pred='no'
    if flag:
        return pred
    else:
        if pred=='yes':
            return 'no'
        else:
            return 'yes'

In [29]:
def generate_basic_ans(layout,img_id,ques_id,thresh=0.2):
    arg=layout[0]['argument'][0]
    scenic_result=load_pkl(os.path.join(GQA_PATH,'meta_one-all-scenic',ques_id+'.pkl'))
    info=scenic_result[arg]
    scores=[]
    scores=[i['score'] for i in info if i['score']>=thresh]
    if len(scores)==0:
        return 'no'
    if max(scores)>=thresh:
        return 'yes'
    else:
        return 'no'

In [30]:
def filter_position(bb,pos,input_image,showing=False):
    if showing:
        vis_no_score([bb],input_image,pos)
    if pos=='left':
        if bb[0]<0.4:
            return 'yes'
        else:
            return 'no'
    elif pos=='right':
        if bb[0]>0.6:
            return 'yes'
        else:
            return 'no'
    elif pos=='top':
        if bb[1]<0.4:
            return 'yes'
        else:
            return 'no'
    elif pos=='bottom':
        if bb[1]>0.6:
            return 'yes'
        else:
            return 'no'

In [31]:
def vis_no_score(bbox,input_image,text=None):
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(input_image, extent=(0, 1, 1, 0))
    ax.set_axis_off()
    for i,box in enumerate(bbox):
        cx, cy, w, h = box
        ax.plot([cx - w / 2, cx + w / 2, cx + w / 2, cx - w / 2, cx - w / 2],
                [cy - h / 2, cy - h / 2, cy + h / 2, cy + h / 2, cy - h / 2], 'r')
    if text is not None:
        ax.set_title(text,  fontsize=12)
            
def verify_attr_ans(layout,img_id,ques_id,showing=False):
    attr=layout[-2]['argument'][0]#the last filter module
    input_image=get_gqa_feat(img_id)
    if len(layout)==3:
        _,bbox,_=bbox_generator(layout,0,img_id,ques_id)
    else:
        bbox,cap=phrase_bbox_generator_exist(layout,len(layout)-2,img_id)
    if attr in ['left','right','top','bottom']:
        pred=filter_position(bbox,attr,input_image,showing)
    else:
        pred=filter_binary_color(bbox,input_image,attr,showing)
    return pred

In [32]:
def generate_scene_ans(layout,img_id,ques_id,showing=False):
    attr=layout[-2]['argument'][0]#the last filter module
    input_image=get_gqa_feat(img_id)
    bbox=[0.5,0.5,0.99,0.99]#the whole image
    pred=filter_binary_color(bbox,input_image,attr,showing)
    return pred

In [33]:
def existence_for_all(layout,img_id,ques_id):
    ops=[step['operation'] for step in layout]
    if 'filter' not in ops:
        pred=generate_basic_ans(layout,img_id,ques_id)
    elif 'relocate' not in ops:
        if layout[0]['argument'][0]=='scene':
            pred=generate_scene_ans(layout,img_id,ques_id)
        else:
            pred=verify_attr_ans(layout,img_id,ques_id)
    else:
        pred=relocate_involved_ans(layout,img_id,ques_id)
    return pred

In [34]:
meta_prog=load_pkl('../meta_generated_layout/dep_all_meta_layout.pkl')

In [35]:
#focus on exist now
random.shuffle(names)
vis=0
acc=0.0
colors=defaultdict(int)
materials=defaultdict(int)
shapes=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    new_prog=meta_prog[name]
    if new_prog[-1]['operation']!='query':
        continue
    if len(new_prog[-1]['argument'])==0:
        continue
    query_t=new_prog[-1]['argument'][0]
    if query_t=='color':
        colors[row['answer']]+=1
    elif query_t=='material':
        materials[row['answer']]+=1
    elif query_t=='shape':
        shapes[row['answer']]+=1
    vis+=1
print (vis)

68167


In [36]:
hold_list=[]
hold_list.extend([c for c in colors])
hold_list.extend([c for c in materials])
hold_list.extend([c for c in shapes])
hold_list=set(hold_list)
#print (hold_list)

In [37]:
def separate_prog(layout):
    first_prog=[]
    second_prog=[]
    flag=True#whether meet the first exist module
    flag_idx=0
    inserts=0
    for i ,step in enumerate(layout[:-1]):
        if flag:
            first_prog.append(step)
            if step['operation']=='exist':
                flag=False
                flag_idx=i+1
            continue
        dep=step['dependencies']
        second_prog.append(step)
        while len(dep)>0 and dep[0]<flag_idx-1:
            idx=dep[0]
            insertion=layout[idx]
            second_prog.insert(0,insertion)
            inserts+=1
            dep=insertion['dependencies']
    for step in second_prog:
        dep=step['dependencies']
        if len(dep)>0:
            idx=dep[0]
            if idx>=flag_idx:
                dep[0]-=flag_idx
                dep[0]+=inserts
    return first_prog,second_prog

In [38]:
def print_prog(layout):
    for i,step in enumerate(layout):
        dep=[str(d) for d in step['dependencies']]
        dep=','.join(dep)
        dep='  '+dep
        print('\t','--'.join([str(i),step['operation'],step['argument'][0],dep]))

In [39]:
def ans_logical(layout,img_id,ques_id):
    first_prog,second_prog=separate_prog(layout)
    first_ans=existence_for_all(first_prog,img_id,ques_id)
    second_ans=existence_for_all(second_prog,img_id,ques_id)
    logic=layout[-1]['operation']
    if logic=='and':
        if first_ans=='yes' and second_ans=='yes':
            return 'yes',first_ans,second_ans
        else:
            return 'no',first_ans,second_ans
    elif logic=='or':
        if first_ans=='no' and second_ans=='no':
            return 'no',first_ans,second_ans
        else:
            return 'yes',first_ans,second_ans

In [40]:
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
invalid=[]
acc=0.0
total={}
for k,name in enumerate(names):
    row=gqa_val_q[name]
    if row['types']['structural']!='logical':
        continue
    new_prog=meta_prog[name]
    texts=[' '.join([str(vis),row['question'],row['imageId'],name])]
    for i, step in enumerate(new_prog[:-1]):
        dep=[str(d) for d in step['dependencies']]
        dep=','.join(dep)
        dep='  '+dep
        texts.append('--'.join([str(i),step['operation'],step['argument'][0],dep]))
    texts.append(new_prog[-1]['operation'])
    texts='\n'.join(texts)
    """fs_prog,sd_prog=separate_prog(new_prog)
    print_prog(fs_prog)
    print ('\n')
    print_prog(sd_prog)
    print ('\n')"""
    #pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    try:
        pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
    vis+=1
    total[name]=pred
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('Already finished',vis)
        print ('\tAccuracy',acc*100.0/vis)
        print ('\t',len(invalid))
        
print (vis)
print (len(invalid))
print (acc*100.0/vis)

/root/.cache/torch/hub/ashkamath_mdetr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


Already finished 500
	Accuracy 68.8
	 4
Already finished 1000
	Accuracy 69.2
	 5
Already finished 1500
	Accuracy 68.93333333333334
	 7
Already finished 2000
	Accuracy 68.55
	 10
Invalid caption generation for 2321443 pillow  pillow to the right of pillow 
Invalid caption generation for 2321443 pillow  pillow to the right of pillow 
Already finished 2500
	Accuracy 69.24
	 12
Already finished 3000
	Accuracy 69.03333333333333
	 16
Already finished 3500
	Accuracy 68.62857142857143
	 17
Already finished 4000
	Accuracy 68.875
	 19
Already finished 4500
	Accuracy 68.71111111111111
	 21
Already finished 5000
	Accuracy 68.46
	 26
Already finished 5500
	Accuracy 68.52727272727273
	 32
Already finished 6000
	Accuracy 68.45
	 33
Already finished 6500
	Accuracy 68.24615384615385
	 36
Already finished 7000
	Accuracy 68.47142857142858
	 39
Already finished 7500
	Accuracy 68.2
	 43
Already finished 8000
	Accuracy 68.2375
	 44
Already finished 8500
	Accuracy 68.36470588235294
	 47
Already finished 9000

In [41]:
pkl.dump(total,open('logical.pkl','wb'))

In [41]:
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
invalid=[]
acc=0.0
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    if row['types']['structural']!='logical':
        continue
    new_prog=meta_prog[name]
    if len(new_prog)!=5:
        continue
    texts=[' '.join([str(vis),row['question'],row['imageId'],name])]
    for i, step in enumerate(new_prog[:-1]):
        dep=[str(d) for d in step['dependencies']]
        dep=','.join(dep)
        dep='  '+dep
        texts.append('--'.join([str(i),step['operation'],step['argument'][0],dep]))
    texts.append(new_prog[-1]['operation'])
    texts='\n'.join(texts)
    fs_prog,sd_prog=separate_prog(new_prog)
    print_prog(fs_prog)
    print_prog(sd_prog)
    #pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    try:
        pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
    print (row['question'],row['answer'])
    print (row['imageId'],name)
    for i,step in enumerate(new_prog):
        print ('\t',i,step['operation'],step['argument'])
    print (fs_ans,sd_ans,'Prediction:',pred,'\n')
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('Already finished',vis)
        print ('\tAccuracy',acc*100.0/vis)
        print ('\t',len(invalid))
        
print (vis)
print (len(invalid))
print (acc*100.0/vis)

	 0--select--blanket--  
	 1--exist--?--  0
	 0--select--blanket--  
	 1--select--bed--  
	 2--exist--?--  0
Are there both blankets and beds in the picture? yes
2319473 00355900
	 0 select ['blanket']
	 1 exist ['?']
	 2 select ['bed']
	 3 exist ['?']
	 4 and []
yes yes Prediction: yes 

	 0--select--window--  
	 1--exist--?--  0
	 0--select--window--  
	 1--select--door--  
	 2--exist--?--  0
Are there both a door and a window in the picture? yes
2397563 17135398
	 0 select ['window']
	 1 exist ['?']
	 2 select ['door']
	 3 exist ['?']
	 4 and []
yes yes Prediction: yes 

	 0--select--coach--  
	 1--exist--?--  0
	 0--select--coach--  
	 1--select--employee--  
	 2--exist--?--  0
Are there any coaches or employees? no
2340117 07806226
	 0 select ['coach']
	 1 exist ['?']
	 2 select ['employee']
	 3 exist ['?']
	 4 or []
yes yes Prediction: yes 

	 0--select--plate--  
	 1--exist--?--  0
	 0--select--plate--  
	 1--select--sandwich--  
	 2--exist--?--  0
Do you see either plates or sa

In [40]:
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
invalid=[]
acc=0.0
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    if row['types']['structural']!='logical':
        continue
    new_prog=meta_prog[name]
    if len(new_prog)!=5:
        continue
    texts=[' '.join([str(vis),row['question'],row['imageId'],name])]
    for i, step in enumerate(new_prog[:-1]):
        dep=[str(d) for d in step['dependencies']]
        dep=','.join(dep)
        dep='  '+dep
        texts.append('--'.join([str(i),step['operation'],step['argument'][0],dep]))
    texts.append(new_prog[-1]['operation'])
    texts='\n'.join(texts)
    """fs_prog,sd_prog=separate_prog(new_prog)
    print_prog(fs_prog)
    print ('\n')
    print_prog(sd_prog)
    print ('\n')"""
    #pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    try:
        pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('Already finished',vis)
        print ('\tAccuracy',acc*100.0/vis)
        print ('\t',len(invalid))
        
print (vis)
print (len(invalid))
print (acc*100.0/vis)

Already finished 500
	Accuracy 78.8
	 7
Already finished 1000
	Accuracy 79.5
	 12
Already finished 1500
	Accuracy 79.4
	 19
Already finished 2000
	Accuracy 79.8
	 21
Already finished 2500
	Accuracy 79.52
	 27
Already finished 3000
	Accuracy 79.8
	 33
Already finished 3500
	Accuracy 79.77142857142857
	 39
Already finished 4000
	Accuracy 79.8
	 45
Already finished 4500
	Accuracy 79.71111111111111
	 52
Already finished 5000
	Accuracy 79.72
	 56
Already finished 5500
	Accuracy 79.6
	 61
Already finished 6000
	Accuracy 79.4
	 65
Already finished 6500
	Accuracy 79.36923076923077
	 67
Already finished 7000
	Accuracy 79.32857142857142
	 68
Already finished 7500
	Accuracy 79.22666666666667
	 72
Already finished 8000
	Accuracy 79.3125
	 77
Already finished 8500
	Accuracy 79.37647058823529
	 80
Already finished 9000
	Accuracy 79.53333333333333
	 87
9083
89
79.53319387867445


In [42]:
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
invalid=[]
acc=0.0
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    if row['types']['structural']!='logical':
        continue
    new_prog=meta_prog[name]
    if len(new_prog)!=6:
        continue
    texts=[' '.join([str(vis),row['question'],row['imageId'],name])]
    for i, step in enumerate(new_prog[:-1]):
        dep=[str(d) for d in step['dependencies']]
        dep=','.join(dep)
        dep='  '+dep
        texts.append('--'.join([str(i),step['operation'],step['argument'][0],dep]))
    texts.append(new_prog[-1]['operation'])
    texts='\n'.join(texts)
    """fs_prog,sd_prog=separate_prog(new_prog)
    print_prog(fs_prog)
    print ('\n')
    print_prog(sd_prog)
    print ('\n')"""
    #pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    try:
        pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('Already finished',vis)
        print ('\tAccuracy',acc*100.0/vis)
        print ('\t',len(invalid))
        
print (vis)
print (len(invalid))
print (acc*100.0/vis)

/root/.cache/torch/hub/ashkamath_mdetr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


Already finished 500
	Accuracy 54.6
	 0
Already finished 1000
	Accuracy 55.2
	 0
Already finished 1500
	Accuracy 55.2
	 1
1913
2
54.260324098274964


In [44]:
#focus on exist now
#without the scene related questions
hold_dict={'hposition': 6502, 'name': 47463, 'color': 7790, 
           'material': 1387, 'size': 860, 'place': 2166, 'length': 203}
thresh=0.2
random.shuffle(names)
vis=0
invalid=[]
acc=0.0
for k,name in enumerate(names):
    #if vis>10:
    #    break
    row=gqa_val_q[name]
    if row['types']['structural']!='logical':
        continue
    new_prog=meta_prog[name]
    if len(new_prog)!=7:
        continue
    texts=[' '.join([str(vis),row['question'],row['imageId'],name])]
    for i, step in enumerate(new_prog[:-1]):
        dep=[str(d) for d in step['dependencies']]
        dep=','.join(dep)
        dep='  '+dep
        texts.append('--'.join([str(i),step['operation'],step['argument'][0],dep]))
    texts.append(new_prog[-1]['operation'])
    texts='\n'.join(texts)
    """fs_prog,sd_prog=separate_prog(new_prog)
    print_prog(fs_prog)
    print ('\n')
    print_prog(sd_prog)
    print ('\n')"""
    #pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    try:
        pred,fs_ans,sd_ans=ans_logical(new_prog,row['imageId'],name)
    except:
        invalid.append(name)
    vis+=1
    if pred==row['answer'].strip():
        acc+=1
    if vis%500==0:
        print ('Already finished',vis)
        print ('\tAccuracy',acc*100.0/vis)
        print ('\t',len(invalid))
        
print (vis)
print (len(invalid))
print (acc*100.0/vis)

Already finished 500
	Accuracy 55.4
	 0
Already finished 1000
	Accuracy 58.3
	 1
Already finished 1500
	Accuracy 57.13333333333333
	 1
Already finished 2000
	Accuracy 57.15
	 1
Already finished 2500
	Accuracy 56.52
	 1
Already finished 3000
	Accuracy 56.2
	 2
Already finished 3500
	Accuracy 56.05714285714286
	 2
3766
2
55.86829527349973
